In [1]:
import sys
sys.path.insert(0, './src')

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from src.utils.metric_util import Evaluator

In [3]:
df = pd.read_csv('scaled.csv')
x = df.drop(['Arrhythmia'], axis=1)
y = df['Arrhythmia']
x

,Age,SBP,DBP,BPM,Pro_BNP,CRP,TC,TG,HDL,LDL,...,smoker,drinker,PriorMI,PriorCI,PriorHF,PriorCHD,sex,VWM,PCI,BBB
0,0.428571,0.333333,0.333333,0.333333,0.000000,1.0,0.00,0.0,0.0,0.50,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667,0.000000,1.0
1,0.428571,0.333333,0.166667,0.500000,0.333333,1.0,0.25,0.0,0.0,0.25,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.166667,0.000000,1.0
2,0.428571,0.500000,0.500000,0.166667,0.500000,0.0,0.50,0.5,1.0,0.50,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.166667,0.000000,1.0
3,0.428571,0.500000,0.333333,0.166667,0.000000,0.0,0.50,1.0,0.0,0.50,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667,0.000000,1.0
4,0.428571,0.500000,0.500000,0.333333,0.000000,0.0,0.00,1.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077,0.714286,0.500000,0.500000,0.333333,0.000000,1.0,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.166667,0.571429,0.0
2078,0.428571,0.166667,0.000000,0.000000,0.000000,1.0,0.50,1.0,0.0,0.50,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.666667,1.000000,0.0
2079,0.571429,0.333333,0.333333,0.666667,1.000000,1.0,0.25,1.0,0.0,0.25,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.142857,0.0
2080,0.714286,0.666667,0.166667,0.166667,0.666667,1.0,0.00,0.0,0.0,0.00,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667,1.000000,0.0


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True, stratify=y, test_size=0.3)

In [5]:
from src.config import Config
from src.utils.data_util import get_dataset

In [6]:
#amend position function
def amend_position(position, lower, upper):
    pos = np.clip(position, lower, upper).astype(int)
    if np.all((pos == 0)):
        pos[np.random.randint(0, len(pos))] = 1
    return pos

In [7]:
#fitness function
def fitness_function(solution):
    evaluator = Evaluator(x_train.values, x_test.values, y_train, y_test, solution, Config.CLASSIFIER, Config.DRAW_CONFUSION_MATRIX, Config.AVERAGE_METRIC)
    metrics = evaluator.get_metrics()
    if Config.PRINT_ALL:
        print(metrics)
    return list(metrics.values())

In [8]:
n_features = 43
LOWER_BOUND = [0, ] * n_features
UPPER_BOUND = [1.04, ] * n_features
problem_dict1 = {
    "fit_func": fitness_function,
    "lb": LOWER_BOUND,
    "ub": UPPER_BOUND,
 
    "minmax": Config.MIN_MAX_PROBLEM,
    "obj_weights": Config.OBJ_WEIGHTS,
    "amend_position": amend_position
}

In [9]:
from mealpy.swarm_based.CSA import OriginalCSA

In [10]:
md_csa = OriginalCSA(epoch=20, pop_size=50, p_a=0.23)
best_position, best_fitness= md_csa.solve(problem=problem_dict1)

2023/01/14 11:14:02 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: Solving 4-objective optimization problem with weights: [1 0 0 0].
2023/01/14 11:14:29 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 1, Current best: 0.6096, Global best: 0.6096, Runtime: 14.56191 seconds
2023/01/14 11:14:43 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 2, Current best: 0.6096, Global best: 0.6096, Runtime: 14.01886 seconds
2023/01/14 11:14:57 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 3, Current best: 0.5936, Global best: 0.6096, Runtime: 14.11782 seconds
2023/01/14 11:15:11 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 4, Current best: 0.608, Global best: 0.6096, Runtime: 14.41169 seconds


KeyboardInterrupt: 

In [11]:
best_position

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [117]:
def feature_name(features):
    df1 = pd.read_csv('scaled.csv')
    names = list(df1.columns.values)
    feature = []
    i = 0
    for name in features:
        if name == 1:
            feature.append(names[i])
        else:
            pass
        i+=1
    
    return feature

In [52]:
feature_name(best_position)

['BPM', 'RARL', 'sex']

In [53]:
best_fitness

0.6304

In [106]:
n_features = 43
LOWER_BOUND = [0, ] * n_features
UPPER_BOUND = [1, ] * n_features
problem_dict1 = {
    "fit_func": fitness_function,
    "lb": LOWER_BOUND,
    "ub": UPPER_BOUND,
 
    "minmax": Config.MIN_MAX_PROBLEM,
    "obj_weights": Config.OBJ_WEIGHTS,
    "amend_position": amend_position
}

In [107]:
md_csa = OriginalCSA(epoch=20, pop_size=50)
best_position, best_fitness= md_csa.solve(problem=problem_dict1)

2023/01/12 12:11:12 PM, INFO, mealpy.swarm_based.CSA.OriginalCSA: Solving 4-objective optimization problem with weights: [1 0 0 0].
2023/01/12 12:11:54 PM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 1, Current best: 0.6048, Global best: 0.6048, Runtime: 24.31172 seconds
2023/01/12 12:12:17 PM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 2, Current best: 0.6, Global best: 0.6048, Runtime: 22.85021 seconds
2023/01/12 12:12:41 PM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 3, Current best: 0.6048, Global best: 0.6048, Runtime: 23.20068 seconds
2023/01/12 12:13:05 PM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 4, Current best: 0.6016, Global best: 0.6048, Runtime: 24.64497 seconds
2023/01/12 12:13:31 PM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 5, Current best: 0.6032, Global best: 0.6048, Runtime: 25.37725 seconds
2023/01/12 12:13:54 PM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Ep

In [108]:
feature_name(best_position)

['PCI']

In [109]:
best_fitness

0.6048

In [99]:
from sklearn.model_selection import StratifiedKFold

In [100]:
rfc = RandomForestClassifier(random_state=2, max_depth=37, n_estimators=96, min_samples_leaf=2, n_jobs=-1)

In [101]:
import numpy as np

In [104]:
df = pd.read_csv('encoded_data.csv')
x = np.array(df[list(feature_name(best_position))])
y = np.array(df['Arrhythmia'])
x

array([[2, 1],
       [1, 0],
       [3, 1],
       ...,
       [2, 1],
       [1, 1],
       [4, 1]], dtype=int64)

In [105]:
strat_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
accuracy_list = []
for train_index, test_index in strat_kfold.split(x, y):
    x_train_fold, x_test_fold = x[train_index], x[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    rfc.fit(x_train_fold, y_train_fold)
    preds = rfc.predict(x_test_fold)
    accuracy_test = accuracy_score(preds, y_test_fold)
    accuracy_list.append(accuracy_test)
np.array(accuracy_list).mean()

0.6186579867500921

# Iterrations for different values

In [110]:
#amend position function
def amend_position(position, lower, upper):
    pos = np.clip(position, lower, upper).astype(int)
    if np.all((pos == 0)):
        pos[np.random.randint(0, len(pos))] = 1
    return pos

In [111]:
#fitness function
def fitness_function(solution):
    evaluator = Evaluator(x_train.values, x_test.values, y_train, y_test, solution, Config.CLASSIFIER, Config.DRAW_CONFUSION_MATRIX, Config.AVERAGE_METRIC)
    metrics = evaluator.get_metrics()
    if Config.PRINT_ALL:
        print(metrics)
    return list(metrics.values())

In [113]:
n_features = 43
LOWER_BOUND = [0, ] * n_features
UPPER_BOUND = [1.02, ] * n_features
problem_dict1 = {
    "fit_func": fitness_function,
    "lb": LOWER_BOUND,
    "ub": UPPER_BOUND,
 
    "minmax": Config.MIN_MAX_PROBLEM,
    "obj_weights": Config.OBJ_WEIGHTS,
    "amend_position": amend_position
}

In [180]:
md_csa = OriginalCSA(epoch=20, pop_size=50, p_a=0.23)
best_position, best_fitness= md_csa.solve(problem=problem_dict1)

2023/01/14 12:55:27 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: Solving 4-objective optimization problem with weights: [1 0 0 0].
2023/01/14 12:55:50 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 1, Current best: 0.608, Global best: 0.6304, Runtime: 12.93463 seconds
2023/01/14 12:56:03 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 2, Current best: 0.6016, Global best: 0.6304, Runtime: 13.49920 seconds
2023/01/14 12:56:16 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 3, Current best: 0.624, Global best: 0.6304, Runtime: 12.60752 seconds
2023/01/14 12:56:29 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 4, Current best: 0.6032, Global best: 0.6304, Runtime: 12.99190 seconds
2023/01/14 12:56:43 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Epoch: 5, Current best: 0.624, Global best: 0.6304, Runtime: 14.08613 seconds
2023/01/14 12:56:56 AM, INFO, mealpy.swarm_based.CSA.OriginalCSA: >Problem: P, Ep

In [ ]:
feature_name(best_position)

In [136]:
selected = feature_name(best_position)
selected.append('Arrhythmia')
df_new = df[selected]
df_new

,BPM,Vao,PriorCHD,PCI,Arrhythmia
0,2,0,0,0,0
1,3,1,0,0,0
2,1,1,0,0,0
3,1,0,0,0,0
4,2,1,0,0,0
...,...,...,...,...,...
2077,2,1,0,4,1
2078,0,1,0,7,1
2079,4,1,0,1,1
2080,1,1,0,7,1


In [137]:
df_new.to_csv('FS_V2.csv', index=False)

In [148]:
df_train = pd.read_csv('FS_V2.csv')
x = np.array(df_train.drop(['Arrhythmia'], axis=1))
y = np.array(df_train['Arrhythmia'])
x

array([[2, 0, 0, 0],
       [3, 1, 0, 0],
       [1, 1, 0, 0],
       ...,
       [4, 1, 0, 1],
       [1, 1, 0, 7],
       [2, 0, 0, 4]], dtype=int64)

In [143]:
rfc = RandomForestClassifier(random_state=2, max_depth=37, n_estimators=96, min_samples_leaf=2, n_jobs=-1)

In [144]:
#accuracies = []

In [149]:
strat_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=23)
accuracy_list = []
for train_index, test_index in strat_kfold.split(x, y):
    x_train_fold, x_test_fold = x[train_index], x[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    rfc.fit(x_train_fold, y_train_fold)
    preds = rfc.predict(x_test_fold)
    accuracy_test = accuracy_score(preds, y_test_fold)
    accuracy_list.append(accuracy_test)
np.array(accuracy_list).mean()

0.6027419948472581

In [150]:
accuracies.append(np.array(accuracy_list).mean())

In [151]:
accuracies

[0.6056910195068089, 0.6027419948472581]

In [153]:
import timeit

In [154]:
from sklearn.model_selection import train_test_split

In [155]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, shuffle=True)

In [171]:
x_test.shape

(625, 4)

In [163]:
%timeit rfc.fit(x_train, y_train)

227 ms ± 8.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [172]:
%timeit preds = rfc.predict(x_test)

32.8 ms ± 537 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [173]:
preds.shape

(208,)

In [174]:
y_test.shape

(625,)

In [175]:
%%timeit
rfc.fit(x_train, y_train)
preds = rfc.predict(x_test)

258 ms ± 4.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
